In [ ]:
import numpy as np
import pandas as pd
import nltk
import inflect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re, string, unicodedata
from nltk.tokenize import word_tokenize

In [ ]:
requisicao = ["""HTTP/1.1 200 OK
Cache-Control: private
Content-Length: 609
Content-Type: text/html
X-Powered-By: ASP.NET
Date: Mon, 03 Dec 2018 14:02:42 GMT
Connection: close

<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" />
<meta http-equiv="Content-Language" content="pt" />
<meta http-equiv="Expires" content="0" />
<meta http-equiv="Cache-Control" content="no-cache, must-revalidate" />
<meta http-equiv="Pragma" content="no-cache" />

<script>
</script>"""]

In [ ]:
resposta = ["""POST example.asp HTTP/1.1
Host: example.com.br
Connection: close
Content-Length: 175
Cache-Control: max-age=0
Origin: https://example
Upgrade-Insecure-Requests: 1
Content-Type: application/x-www-form-urlencoded
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
Referer: https:example
Accept-Encoding: gzip, deflate
Accept-Language: pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7
Cookie: 14514362356
txtOptSel=i76568676776

In [ ]:
from nltk.tokenize import word_tokenize
tokenized_req = word_tokenize(str(requisicao))
tokenized_res = word_tokenize(str(resposta))

In [ ]:
def to_lowercase(words): # Converte minusculo
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words): # Remove pontuação
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words): # Replace all interger occurrences with textual representation
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words): # Remove stopwords
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [ ]:
req = normalize(tokenized_req)
res = normalize(tokenized_res)

In [ ]:
vec = TfidfVectorizer()

In [ ]:
X = vec.fit_transform(req)
req = cosine_similarity(X)

In [ ]:
Y = vec.fit_transform(res)
res = cosine_similarity(Y)

In [ ]:
def cosine_similarity (v, w):
    return np.dot(v, w)/np.sqrt(np.dot(v,v)*np.dot(w,w))

In [ ]:
def text_similarity(iid, features=res, text=res, n_text=1):
    similarity = [[cosine_similarity(features[iid],feature), int(i)] for i, feature in enumerate(features)]
    similarity = np.array(sorted(similarity, key=lambda sim: sim[0], reverse=True))
    return [[text[y], similarity[x, 0]] for x, y in enumerate(np.int0(similarity[1:,1]), 1)][:n_text]

In [ ]:
print('texto',requisicao[0])
for t, s in text_similarity(0):
    print('S:{} '.format(round(s, 2)))

In [ ]:
#FÓRMULA MATEMÁTICA:
import math
3/(math.sqrt(7)*math.sqrt(4))

In [ ]:
from collections import Counter

# count ocorrencia de palavras
a_vals = Counter(tokenized_req)
b_vals = Counter(tokenized_res)

# C0nvertendo word-vectors
words  = list(a_vals.keys() | b_vals.keys())
a_vect = [a_vals.get(word, 0) for word in words]
b_vect = [b_vals.get(word, 0) for word in words]

# COsseno
len_a  = sum(av*av for av in a_vect) ** 0.5
len_b  = sum(bv*bv for bv in b_vect) ** 0.5
dot    = sum(av*bv for av,bv in zip(a_vect, b_vect))
cosine = dot / (len_a * len_b) 

In [ ]:
cosine

In [ ]:
sim = np.dot(a_vect, b_vect) / np.sqrt(np.dot(a_vect, a_vect) * np.dot(b_vect, b_vect))

In [ ]:
sim